In [30]:
# Import dependencies
import subprocess
import json
import os
from dotenv import load_dotenv
from web3 import Web3
import bit
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from eth_account import Account 

In [5]:
#create web3 object
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [6]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")
numderive = 3

In [7]:
# Import constants.py and necessary functions from bit and web3
from constants import *

In [8]:
# Create a function called `derive_wallets`
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

In [9]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == BTCTEST:
        return bit.PrivateKeyTestnet(priv_key) 
    elif coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    else:
        print('Error: Invalid coin selection. Please use ETH or BTCTEST')

In [10]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.

def create_tx(coin, account, to, amount):
    if coin == ETH:
        
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": to, "value": amount}
        )
        return {
            "from": account.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
    elif coin == BTCTEST:
        return bit.PrivateKeyTestnet.prepare_transaction(account.address, [(to,amount,BTC)])
    else:
        print('Error: Invalid coin selection. Please use ETH or BTCTEST')

In [11]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.

def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(account, to, amount)
        signed_tx = account.sign_transaction(raw_tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        raw_tx = create_tx(coin,account,to,amount)
        signed_tx = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed_tx)
    else:
        print('Error: Invalid coin selection. Please use ETH or BTCTEST')

In [12]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    ETH: derive_wallets(coin=ETH),    
    BTCTEST: derive_wallets(coin=BTCTEST),
}

In [22]:
account_one = priv_key_to_account(BTCTEST, 'cUrJMxeWVhuspKac36ZwQXruccRg84e8SjuahNm6yuA72qEy9RxX')

In [26]:
account_two = 'tb1q8f9exefyvjrmpne82mtk0ehd8vedrhdvy2qalc'
send_tx(BTCTEST, account_one, account_two, 0.0001)


In [28]:
from web3.middleware import geth_poa_middleware

w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [45]:
ETH_1 = priv_key_to_account(ETH,'0xd8e2e0a98d054c0ad29cc94f47ae61f4fb4ce41c2569e1561b4e8b626e6f5b87')
ETH_2 = '0x235355B881e6201BEDAADD8e2BFDF0cB4027e914'

In [46]:
send_tx(coin=ETH, account=ETH_1, to=ETH_2, amount=1)

TypeError: create_tx() missing 1 required positional argument: 'amount'